# Studierendengenerator Notebook
Thomas Delissen - September 2023

Ziel dieses Notebook ist um Beispieldaten zu generieren für eine Hackathon. 

Input: 
- Lehrveranstaltungen von eine Studiengang - lvs.txt
- vornamen.csv
- nachnamen.csv
- (schulabschlusse: Hardcoded in dieses Notebook)

Output: 
- Liste von Studierenden (Absolventen)
- Liste von noten von diese Studierenden pro LV

Die Studierenden sollten alle LVs abgeschlossen haben. Die Noten werden random generiert, sie sollten aber einigermaßen realistisch sein; das bedeutet das wenn die Studierenden gut sein in Grundlagen des Programmierens, sie auch gut sind in Programmieren für Data Scientist. 

Ansatz: 
1. Wir werden zuerst die Tabelle von Studierenden generieren. Jede Studierende bekommt eine unique ID, Name, Gender, und random Schulabschluss. 
2. Abhängig von Schulabschluss bekommen sie dann eine Random "Güte". Diese wird in 4 variable gespeichert. Eine für die allgemeine Intelligenz, dann eine pro Kategorie LV (Allgemein, Statistik, Programmieren). 0 ist schlecht, 1 ist gut. 
3. Dann werden pro Student alle Noten generiert. Diese werden pro LV random generiert (normal verteilt), wobei die Intelligenz der Student einfluss hat auf die höhe der Note.

In [24]:
import pandas as pd
import numpy as np
import random


## Schritt 1. Generier 400 studenten

In [7]:
vornamen = pd.read_csv("Vornamen.csv")
nachnamen = pd.read_csv("nachnamen.csv")
vornamen.head()
#nachnamen.head()

,Name,Typ,Geschlecht
0,Aabed,Vorname,M
1,Aada,Vorname,F
2,Aaden,Vorname,M
3,Aadi,Vorname,M
4,Aadit,Vorname,M


In [47]:
def student(vornamen,nachnamen):
    # diese funktion generiert ein Student, als eine Liste von: 
    # vorname, nachname, geschlecht, schulabschluss
    # input sein zwei dataframes, welche alle mögliche vornamen und nachnamen beinhalten
    # schulabschluss ist hardcoded in diese Funktion

    index = random.randint(0,len(vornamen)-1)
    vorname = vornamen.iloc[index]["Name"]
    # Verbesserungsmöglichkeiten: Geschlecht ist jetzt 50 / 50, was natürlich nicht realistisch ist.
    geschlecht = vornamen.iloc[index]["Geschlecht"]
    nachname = nachnamen["Nachname"].sample().values[0]
    # Schulabschluss werde ich verteilen über HTL, HAK, AHS, Sonstige. Ich habe gesampled aus BDS2022 für die Verteilung:
    # HTL: 12 / 32. HAK: 8 / 32. AHS: 6 / 32. Sonstige: 6 / 32
    # 0 - 0.375 = HAK. 0.375 - 0.625 = HAK. 0.625 - 0.8125 = AHS. Rest = Sonstige
    randnr = random.random()
    schulabschluss = ""
    if randnr < 0.375:
        schulabschluss = "HTL"
    elif randnr < 0.625:
        schulabschluss = "HAK"
    elif randnr < 0.8125:
        schulabschluss = "AHS"
    else: 
        schulabschluss = "Sonstige"
    
    # jetzt generieren wir die "gute" dieser student, in vier variable. Diese sollten random normal verteilt sein.
    # 0 ist schlecht, 1 ist gut. Es soll nicht kleiner als 0 oder großer als 1 sein, deswegen verwende ich clip
    randnr = np.clip(np.random.normal(0.5, 0.25),0,1)
    gute = randnr
    if schulabschluss == "HTL":
        gute = np.clip(gute + 0.2,0,1)
    elif schulabschluss == "HAK":
        gute = np.clip(gute + 0.1,0,1)
    elif schulabschluss == "AHS":
        gute = np.clip(gute + 0.05,0,1)
    else:
        gute = np.clip(gute - 0.1,0,1)

    # Das gleiche mach ich jetzt für die 3 Teilbereiche. Sie werden aber auch stark beeinflusst vom generale gute
    randnr = np.clip(np.random.normal(gute, 0.25),0,1)
    allgemein = np.clip(randnr + 0.2,0,1)
    randnr = np.clip(np.random.normal(gute, 0.10),0,1)
    statistik = np.clip(randnr,0,1)
    randnr = np.clip(np.random.normal(gute, 0.05),0,1)
    programmieren = np.clip(randnr-0.2,0,1)

    if schulabschluss == "HTL":
        statistik = np.clip(statistik+0.1,0,1)
        programmieren = np.clip(programmieren+0.2,0,1)

    return [vorname,nachname,geschlecht,schulabschluss,gute,allgemein,statistik,programmieren]
#    print(vorname + " " + nachname + " " + geschlecht + " " + schulabschluss)



In [48]:
print(student(vornamen,nachnamen))

['Eghosa', 'Rauter', 'M', 'AHS', 0.36956626564201805, 0.4062656629190726, 0.29426690895676877, 0.1042186524418614]


## Noten generieren
Jetzt gehen wir die Noten generieren, zuerst als eine score zwischen 0 und 1, diese konvertiere ich dann später auf der range 4 bis 1, wobei 1 das beste ist, und 4 das schlechtste. 5 gibt es nicht, weil wir die entscheidung gemacht haben nur die "passing" grade zu zeigen. 
Mean der noten soll so rundum die 2 liegen, ist mein bauchgefühl, weil wir immer zu net sein mit benoten. 

In [91]:
def get_note(kategorie, gute, allgemein, statistik, programmieren):
    # diese Funktion gibt eine note zwischen 4 und 1 zurück (integer), wobei 1 die beste Note ist.
    # Kategorie: 1 = allgemein, 2 = Statistik, 3 = programmieren
    note = 0
    randnr = np.random.normal(gute, 0.35) # man kann gute und schlechte Tage haben
    if kategorie == 1:
        note = (randnr + allgemein) / 2
    elif kategorie == 2: 
        note = (randnr + statistik) / 2
    else:
        note = (randnr + programmieren) / 2
    # jetzt soll die Note zwischen 0 und 1 sein; in ausnahmefalle großer als 1. 

    schulnote = -1 # wenn das rauskommt ist etwas schief gegangen
    # TWEAKING: Ich habe die thresholds angepasst, so das weniger oft eine 4 rauskommt, und ofter 2 er und 1 er
    if note < 0.10 :
        schulnote = 4
    elif note < 0.4 :
        schulnote = 3
    elif note < 0.70 :
        schulnote = 2
    else:
        schulnote = 1
    return schulnote
        


## Finale: Generieren von gesamt records, und ausschreiben zu CSV
Zuerst generieren wir ein Dataframe mit studenten. ID werde ich einfach aufzahlen, nichts komplexes. Variablen ab "gute" werden wir nicht exportieren, die sind nur für interne verwendung, um die Noten zu bestimmen

In [67]:
menge_studenten = 400
id = 0
studenten = []
for i in range(menge_studenten):
    studenten.append([id]+student(vornamen,nachnamen))
    id = id+1
studentenframe = pd.DataFrame(studenten,columns = ["stud_id","vorname","nachname","geschlecht","schulabschluss","gute","allgemein","statistik","programmieren"])
studentenframe.head()

,stud_id,vorname,nachname,geschlecht,schulabschluss,gute,allgemein,statistik,programmieren
0,0,Aliye,Pucher,F,HTL,0.299832,0.218677,0.407495,0.330274
1,1,Jennifer-Alexandra,Steindl,F,Sonstige,0.587649,1.000000,0.514216,0.378193
2,2,Jehudit,Hoffmann,F,AHS,0.351863,0.696412,0.398271,0.134583
3,3,Love,Wurm,M,HTL,0.509851,0.702938,0.607280,0.519547
4,4,Kadirhan,Pfeiffer,M,AHS,0.257515,0.481299,0.238530,0.052359


In [92]:
lvs = pd.read_csv("LVs.txt",sep=";")
lvs["lv_id"] = range(1, len(lvs) + 1)
lvs.head()

,Semester,LV Name,ECTS,Kategorie,lv_id
0,1,Einführung in die Datenanalyse,5,1,1
1,1,Einführung in die Ökonomie,2,1,2
2,1,Grundlagen der Mathematik,5,2,3
3,1,Grundlagen der Statistik,5,2,4
4,1,Grundlagen des Programmierens,5,3,5


In [93]:
# Now we will loop through the Students, and for each Student, we will loop through the LVs. 
# For each Student LV combination, we will generate a grade using the function defined above. 

noteliste = []
for i in range(0,len(studentenframe)):
    stud_id = studentenframe.iloc[i]["stud_id"]
    gute = studentenframe.iloc[i]["gute"]
    allgemein = studentenframe.iloc[i]["allgemein"]
    statistik = studentenframe.iloc[i]["statistik"]
    programmieren = studentenframe.iloc[i]["programmieren"]
    for j in range(0,len(lvs)):
        lv_id = lvs.iloc[j]["lv_id"]
        kategorie = lvs.iloc[j]["Kategorie"]
        note = get_note(kategorie, gute, allgemein, statistik, programmieren)
        noteliste.append([stud_id,lv_id,note])

print(len(studentenframe))
print(len(lvs))
print(len(noteliste))


400
22
8800


### Export
Alle Daten sind jetzt synthetisiert, wir sollen sie nur noch ausschreiben. lvs und noteliste können so ausgeschrieben werden, vom studentenframe sollten noch die "interne" spalten gedropt werden. 

In [94]:
studentenoutput = studentenframe[["stud_id","vorname","nachname","geschlecht","schulabschluss"]]
studentenoutput.to_csv("studierenden.csv",index=False)

In [95]:
lvs.to_csv("lvliste.csv",index=False)

In [96]:
notenframe = pd.DataFrame(noteliste,columns=["stud_id","lv_id","note"])
notenframe.to_csv("noteliste.csv",index=False)